In [1]:
# minimal module imports
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayer
from arcgis.geometry import Geometry
import pandas as pd
import os
import importlib

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy
    
import sys
sys.path.append('../src')

app_id = 'c9fa4c15-b9ab-46d4-bcd9-a3eb33dd0ff8'
hash_token = '919745fda981ddccb3ffc4f07863480e624cfa3d'

raw_gdb = os.path.abspath(r'../data/raw/raw.gdb')
int_gdb = os.path.abspath(r'../data/interim/interim.gdb')

dest_fc = os.path.join(int_gdb, 'pdx_businesses_home_goods')
dest_id_fld = 'LOCNUM'
origin_fc = os.path.join(raw_gdb, 'block_groups')
origin_id_fld = 'ID'

from inrix import TAZ
taz = TAZ(app_id, hash_token)
%load_ext autoreload
%autoreload 2

In [2]:
df_pop_cent = GeoAccessor.from_featureclass(dest_fc)

In [12]:
def get_dest_df(geom, locnum):
    dest_df = taz.get_trip_destination_spatial_dataframe(geom.y, geom.x, '100m')
    if dest_df is not None:
        dest_df = dest_df[['travelDistanceMiles', 'travelTimeMinutes', 'SHAPE']].copy()
        dest_df.columns = ['travel_distance_miles', 'travel_time_minutes', 'SHAPE']
        dest_df['store_locnum'] = locnum
        dest_df['x'] = dest_df.SHAPE.apply(lambda geom: geom.x)
        dest_df['y'] = dest_df.SHAPE.apply(lambda geom: geom.y)
        return dest_df
    else:
        return None

In [13]:
df = get_dest_df(geom, locnum)
df.head()

,travel_distance_miles,travel_time_minutes,SHAPE,store_locnum,x,y
0,0.616400,2.450000,"{""x"": -122.437109, ""y"": 45.513666, ""spatialRef...",560254781,-122.437109,45.513666
1,0.623856,3.100000,"{""x"": -122.432196, ""y"": 45.499133, ""spatialRef...",560254781,-122.432196,45.499133
2,1.720576,7.233333,"{""x"": -122.47537, ""y"": 45.50476, ""spatialRefer...",560254781,-122.475370,45.504760
3,0.150993,9.050000,"{""x"": -122.440825, ""y"": 45.504481, ""spatialRef...",560254781,-122.440825,45.504481
4,0.884832,6.400000,"{""x"": -122.424017, ""y"": 45.508376, ""spatialRef...",560254781,-122.424017,45.508376


In [19]:
%%time
for idx, (locnum, geom) in df_pop_cent[['LOCNUM', 'SHAPE']].iterrows():
    new_df = get_dest_df(geom, locnum)
    if new_df is not None:
        if idx == 0:
            dest_df = new_df
        else:
            dest_df = dest_df.append(new_df)

Wall time: 16.3 s


In [20]:
dest_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65430 entries, 0 to 812
Data columns (total 6 columns):
travel_distance_miles    65430 non-null float64
travel_time_minutes      65430 non-null float64
SHAPE                    65430 non-null geometry
store_locnum             65430 non-null object
x                        65430 non-null float64
y                        65430 non-null float64
dtypes: float64(4), geometry(1), object(1)
memory usage: 3.5+ MB


In [21]:
dest_df.head()

,travel_distance_miles,travel_time_minutes,SHAPE,store_locnum,x,y
0,2.334491,3.800000,"{""x"": -122.992346, ""y"": 44.983553, ""spatialRef...",632189163,-122.992346,44.983553
1,6.059610,10.800000,"{""x"": -123.039513, ""y"": 44.942453, ""spatialRef...",632189163,-123.039513,44.942453
2,13.189221,18.983333,"{""x"": -123.091354, ""y"": 44.883755, ""spatialRef...",632189163,-123.091354,44.883755
3,13.140133,17.083333,"{""x"": -122.879561, ""y"": 45.148948, ""spatialRef...",632189163,-122.879561,45.148948
4,7.090464,21.750000,"{""x"": -123.077107, ""y"": 44.956894, ""spatialRef...",632189163,-123.077107,44.956894


In [23]:
dest_df.spatial.set_geometry('SHAPE')
dest_df.spatial.validate()

True

In [17]:
dest_df.to_csv('../data/interim/raw_trips_pdx_home_goods.csv')

In [26]:
dest_df.spatial.to_featureclass('../data/interim/interim.gdb/raw_trips_pdx_home_goods')

KeyError: 'spatialReference'